In [1]:
#params = []
#params.append(["train_size", "K", "learn_rate", "iterations", "w", "error"])

In [2]:
import numpy as np
import csv
import math

train_file = 'train.csv'
test_file  = 'test.csv'
soln_file  = 'user_median.csv'

train_size = .9
K = 5
learn_rate = 0.0000001
iterations = 5
w = .2
weight_user_median = .8
penalty = .02


#load CSV
train_data = {}
test_data = {}
ndata = 0
with open(train_file, 'r') as train_fh:
    ndata = sum(1 for row in train_fh) - 1
    
splitting = np.random.binomial(1, train_size, size=ndata)


In [3]:
with open(train_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    
    for i, row, in enumerate(train_csv):
        user   = row[0]
        artist = row[1]
        plays  = row[2]

        if splitting[i]: 
            if not user in train_data:
                train_data[user] = {}
            train_data[user][artist] = int(plays)
            
        else:
            if not user in test_data:
                test_data[user] = {}

            test_data[user][artist] = int(plays)
              
# Compute the global median and per-user median.
plays_array  = []
user_medians = {}
for user, user_data in train_data.iteritems():
    user_plays = []
    for artist, plays in user_data.iteritems():
        plays_array.append(plays)
        user_plays.append(plays)

    user_medians[user] = np.median(np.array(user_plays))
global_median = np.median(np.array(plays_array))

In [4]:
np.random.rand() = 10

user_mat = []
artist_mat = []


artist_plays = {}
for k in range(K):
    print "\r", k,
    user_mat.append({})
    artist_mat.append({})
    
    for user, user_data in train_data.iteritems():
        if not user in user_mat[k]:
            user_mat[k][user] = np.random.rand()
        for artist, plays in user_data.iteritems():
            if not artist in artist_plays:
                artist_plays[artist] = []
                artist_mat[k][artist] = np.random.rand()
            artist_plays[artist].append(plays)

print len(user_mat[1])


9 233286


In [5]:
artist_meds = {}
for artist in artist_plays.keys():
    artist_meds[artist] = np.median(np.array(artist_plays[artist]))

 

In [6]:
i = 10
test_artist = artist_plays.keys()[i]

print artist_meds[test_artist]


116.0


In [16]:
artist_mat[0][artist_mat[0].keys()[0]]

[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]

In [15]:
user_mat[0][user_mat[k].keys()[0]]

[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]

In [9]:
def predict(user, artist):
    return sum([user_mat[k][user] * artist_mat[k][artist] for k in range(K)])


def modify_prediction(user, artist, prediction):
    user_bias = user_medians[user] - global_median
    return w*prediction + (1-w)*(artist_meds[artist] + user_bias)

def modify_prediction2(user, artist, prediction):
    return (1 - weight_user_median)*prediction + weight_user_median*(user_medians[user])

def train(data):

    for k in range(K):
        user_feature = user_mat[k]
        artist_feature = artist_mat[k]
        
        for j, (user, user_data) in enumerate(train_data.iteritems()):
            if j % 10000 == 0:
                print "\r", j,k,

            for artist, plays in user_data.iteritems(): 
                
                for i in range(iterations):
                    print type(predict(user, artist))
                    user_feature[user] += learn_rate * artist_feature[artist] * (plays - predict(user, artist))
                    artist_feature[artist] += learn_rate * user_feature[user] * (plays - predict(user, artist))
    print user_feature[user]
           
train(train_data)

0 0 <type 'float'>
<type 'list'>
<type 'int'>


TypeError: can't multiply sequence by non-int of type 'list'

In [ ]:
i = 31
x = user_feats.keys()[i]
print user_feats[x]


In [ ]:

def calc_error(data):
    errors = []
    errors2 = []
    pred = 0
    for j, (user, user_data) in enumerate(data.iteritems()):
        if j % 10000 == 0:
            print "\r", j,
        for artist, plays in user_data.iteritems():
            pred = predict_plays(user,artist)
                
            newpred = modify_prediction(user,artist,pred)
            error = abs(int(plays) - newpred)
            errors.append(error)

            newpred2 = modify_prediction2(user,artist,pred)
            error2 = abs(int(plays) - newpred2)
            errors2.append(error2)


    return (sum(np.array(errors))/len(errors), sum(np.array(errors2))/len(errors2))
        
error = calc_error(test_data)

print "Error: ", error


In [ ]:
params.append([train_size, K, learn_rate, iterations, w, weight_user_median, penalty, error])

In [ ]:
print params